# Importacao das libs

**ATENCAO:** *UTILIZE PYTHON __3.11__ OU SUPERIOR!*

In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import camelot

# Puxa o endereco das tabelas

In [2]:
# url da Anvisa
url = "https://www.gov.br/anvisa/pt-br/setorregulado/regularizacao/medicamentos/medicamentos-de-referencia/lista-de-medicamentos-de-referencia"

# instanciando o firefox
driver = webdriver.Firefox()
driver.get(url)

sleep(10)

# buscando a lista de links
xpath = "//div[@id='parent-fieldname-text']//ul"
element = driver.find_element(By.XPATH, xpath)
html_content = element.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')

# separando os links
tableList = list()
for el in soup.find_all(name='a'):
    link = el.get('href')
    if 'https' in  link:
        tableList.append(link)

# encerra o firefox
driver.quit()


In [3]:
# para conferir a lista com o endereco das tabelas
tableList

['https://www.gov.br/anvisa/pt-br/setorregulado/regularizacao/medicamentos/medicamentos-de-referencia/arquivos/lista-a-incluidos-01032024.pdf',
 'https://www.gov.br/anvisa/pt-br/setorregulado/regularizacao/medicamentos/medicamentos-de-referencia/arquivos/lista-a-excluidos-01032024.pdf',
 'https://www.gov.br/anvisa/pt-br/setorregulado/regularizacao/medicamentos/medicamentos-de-referencia/arquivos/lista-b-incluidos-01032024.pdf',
 'https://www.gov.br/anvisa/pt-br/setorregulado/regularizacao/medicamentos/medicamentos-de-referencia/arquivos/lista-b-excluidos-01032024.pdf']

# Define as funcoes usados posteriormente

### funcao destinada a filtrar a tabela

In [4]:
# filtro que funciona:
def filterTable(temp_table):
    for row in range(0, len(temp_table.df)):
        if temp_table.df.at[row, 3] != '' and '/' not in temp_table.df.at[row, 3]:
            for col in range(0, len(temp_table.df.columns)):
                if temp_table.df.at[row, col] == '':
                    # print(f'{temp_table.df.at[row-1, col]} {temp_table.df.at[row+1, col]}')
                    temp_table.df.at[row,col] = f'{temp_table.df.at[row-1, col]} {temp_table.df.at[row+1, col]}'

    for row in range(0, len(temp_table.df)):
        if temp_table.df.at[row, 3] == '' or '/' in temp_table.df.at[row, 3]:
            temp_table.df = temp_table.df.drop(index=row)
        
    temp_table.df = temp_table.df.reset_index(drop=True)
    
    return temp_table

In [ ]:
# filtro novo, nao funciona
def filterTable2(temp_table):
    for row in range(0, len(temp_table.df)):
        if temp_table.df.at[row, 3] == '':
            for col in range(0, len(temp_table.df.columns)):
                if temp_table.df.at[row, col] != '':
                    newData = list()
                    rw = 1
                    while True:
                        if temp_table.df.at[row+rw, 3] == '':
                            rw += 1
                        else:
                            break
                    for temp_row in range(0, rw+rw+1):
                        newData.append(temp_table.df.at[temp_row, col])
                        temp_table.df.at[temp_row, col] = ''
                    
                    print(' '.join(newData))
                    temp_table.df.at[row+rw, col] = ' '.join(newData)
    
    return temp_table

### funcao principal para analise das tabelas

In [10]:
def getDatasOfTable(ntable):
    link_pdf = tableList[ntable]
    tables = camelot.read_pdf(link_pdf, flavor='stream', pages='all', flag='text')

    for pg in range(0, len(tables)):
        table = tables[pg]
        if pg == 0:
            match ntable:
                case 0:
                    table.df = table.df[2:].reset_index(drop=True)
                case 1:
                    table.df = table.df[4:].reset_index(drop=True)
                case 3:
                    table.df = table.df[4:].reset_index(drop=True)

        table = filterTable(table)
        tables[pg].df = table.df

    return tables


# Leitura das tabelas

In [11]:
# declara uma tabela
# tabela A incluidos
myTable = getDatasOfTable(0)

In [13]:
# exibi uma das paginas da tabela
display(myTable[10].df)

,0,1,2,3,4,5,6
0,cetoprofeno,Cristália,CETOPROFENO,102980320,100mg,pó liof sol inj,09/12/2022
1,cetoprofeno,Sanofi-Aventis,PROFENID RETARD,183260360,200mg,com lib retard,12/11/2012
2,cetoprofeno,Eurofarma,BICERTO IV,100431313,1mg/mL,sol infus,18/02/2022
3,cetoprofeno (lisinato),Aché,ARTROSIL,105730128,160mg,cap,12/11/2012
4,cetoprofeno (lisinato),Aché,ARTROSIL,105730128,320mg,cap,12/11/2012
5,cetotifeno (fumarato),União Química,OCTIFEN,104971358,"0,250 mg/mL",sol oft,27/12/2019
6,cetrorrelix (acetato),Merck S/A,CETROTIDE,100890369,"0,25mg",pó liof inj,19/01/2018
7,ciclesonida,Moksha,OMNARIS,164250005,50 mcg/dose,sus spr nas,12/11/2012
8,ciclobenzaprina (cloridrato),Apsen Farmacêutica,MIOSAN,101180129,5mg,com rev,12/11/2012
9,ciclobenzaprina (cloridrato),Apsen Farmacêutica,MIOSAN,101180129,10mg,com rev,12/11/2012
